# BTree

In [1]:
class BTreeNode:
    def __init__(self, t):
        self.n = 0
        self.key = [None] * (2*t-1)
        self.c = [None] * (2*t)
        self.leaf = True

In [ ]:
class BTree:
    def __init__(self, t):
        self.t = t
        self.root = BTreeNode(t)
    
    def disk_read(self, x):
        pass
    
    def disk_write(self, x):
        pass
    
    def split_child(self, x, i):
        t = self.t
        z = BTreeNode(t)
        y = x.c[i]
        z.leaf = y.leaf
        z.n = t - 1
        for j in range(t-1):
            z.key[j] = y.key[j+t]
        if not y.leaf:
            for j in range(t):
                z.c[j] = y.c[j+t]
        y.n = t - 1
        for j in range(x.n, i-1, -1):
            x.c[j+1] = x.c[j]
        x.c[i+1] = z
        for j in range(x.n-1, i-2, -1):
            x.key[j+1] = x.key[j]
        x.key[i] = y.key[t-1]
        x.n += 1
        self.disk_write(x)
        self.disk_write(y)
        self.disk_write(z)
    
    def insert(self, k):
        t = self.t
        r = self.root
        if r.n == 2*t-1:
            s = BTreeNode(t)
            self.root = s
            s.leaf = False
            s.n = 0
            s.c[0] = r
            self.split_child(s, 0)
            self.insert_nonfull(s, k)
        else:
            self.insert_nonfull(r, k)
    
    def insert_nonfull(self, x, k):
        t = self.t
        i = x.n - 1
        if x.leaf:
            while i >= 0 and k < x.key[i]:
                x.key[i+1] = x.key[i]
                i -= 1
            x.key[i+1] = k
            x.n += 1
            self.disk_write(x)
        else:
            while i >= 0 and k < x.key[i]:
                i -= 1
            i += 1
            self.disk_read(x.c[i])
            if x.c[i].n == 2*t-1:
                self.split_child(x, i)
                if k > x.key[i]:
                    i += 1
            self.insert_nonfull(x.c[i], k)
    
    def minimum(self):
        r = self.root
        def _minimum(x):
            if x is None:
                return None
            self.disk_read(x.c[0])
            if x.n > 0 and x.c[0] is not None:
                return _minimum(x.c[0])
            return x.key[0]
        if r.n == 0:
            return None
        return _minimum(r)
    
    def maximum(self):
        r = self.root
        def _maximum(x):
            if x is None:
                return None
            self.disk_read(x.c[x.n])
            if x.n > 0 and x.c[x.n] is not None:
                return _maximum(x.c[x.n])
            return x.key[x.n-1]
        if r.n == 0:
            return None
        return _maximum(r)

    def predecessor(self, k):
        r = self.root
        def _predecessor(x):
            if x is None:
                return None
            for i in range(x.n-1, -1, -1):
                if k > x.key[i]:
                    self.disk_read(x.c[i+1])
                    c = _predecessor(x.c[i+1])
                    if c is None:
                        return x.key[i]
                    return max(c, x.key[i])
            self.disk_read(x.c[0])
            return _predecessor(x.c[0])
        if r.n == 0:
            return None
        return _predecessor(r)
    
    def successor(self, k):
        r = self.root
        def _successor(x):
            if x is None:
                return None
            for i in range(x.n):
                if k < x.key[i]:
                    self.disk_read(x.c[i])
                    c = _successor(x.c[i])
                    if c is None:
                        return x.key[i]
                    return min(c, x.key[i])
            self.disk_read(x.c[x.n])
            return _successor(x.c[x.n])
        if r.n == 0:
            return None
        return _successor(r)
    
    def search(self, k):
        r = self.root
        def _search(x, k):
            i = 0
            while i <= x.n-1 and k > x.key[i]:
                i += 1
            if i <= x.n-1 and k == x.key[i]:
                return (x, i)
            elif x.leaf:
                return None
            else:
                self.disk_read(x.c[i])
                return _search(x.c[i], k)
        if r.n == 0:
            return None
        return _search(r, k)